# ADM - HW 3

# 1. Data Collection

In [1]:
import functions 

from bs4 import BeautifulSoup
import requests
import os 
import time
import pandas as pd 
import csv 
import nltk
import calendar
from nltk.corpus import stopwords
import numpy as np
from collections import Counter
from functools import reduce
import pickle 

## 1.1 Get the list of master's degree courses

In [ ]:
# se to true to install bs4 & selenium
if False: 
    pip install bs4;
    pip install selenium

First of all, we get a list of all the master's degree courses we need.

In [2]:
def extract_masters(this_url):
  result_url = requests.get(this_url)
  result_soup = BeautifulSoup(result_url.text)
  result_links = result_soup.find_all('a', {'class':'courseLink'})
  result_list=[]
  for item in result_links:
    result_list.append((item['href'], item.text))

  return result_list

We notice that the url of th various pages is *https://www.findamasters.com/masters-degrees/msc-degrees/?PG=* followed by the number of the page in the MSc courses catalogue we're in. 

In [3]:
links = []
for page in range(1, 401):
    url = 'https://www.findamasters.com/masters-degrees/msc-degrees/?PG='+str(page)
    links += extract_masters(url)
    print(page, len(links))

1 15
2 30
3 45
4 60
5 75
6 90
7 105
8 120
9 135
10 150
11 165
12 180
13 195
14 210
15 225
16 240
17 255
18 270
19 285
20 300
21 315
22 330
23 345
24 360
25 375
26 390
27 405
28 420
29 435
30 450
31 465
32 480
33 495
34 510
35 525
36 540
37 555
38 570
39 585
40 600
41 615
42 630
43 645
44 660
45 675
46 690
47 705
48 720
49 735
50 750
51 765
52 780
53 795
54 810
55 825
56 840
57 855
58 870
59 885
60 900
61 915
62 930
63 945
64 960
65 975
66 990
67 1005
68 1020
69 1035
70 1050
71 1065
72 1080
73 1095
74 1110
75 1125
76 1140
77 1155
78 1170
79 1185
80 1200
81 1215
82 1230
83 1245
84 1260
85 1275
86 1290
87 1305
88 1320
89 1335
90 1350
91 1365
92 1380
93 1395
94 1410
95 1425
96 1440
97 1455
98 1470
99 1485
100 1500
101 1515
102 1530
103 1545
104 1560
105 1575
106 1590
107 1605
108 1620
109 1635
110 1650
111 1665
112 1680
113 1695
114 1710
115 1725
116 1740
117 1755
118 1770
119 1785
120 1800
121 1815
122 1830
123 1845
124 1860
125 1875
126 1890
127 1905
128 1920
129 1935
130 1950
131 1965
1

We want to verify we retrieved all of the needed urls, so we check for the length of the list

In [5]:
print(len(links))

6000


It's the expected length, so we can create and write the required *.txt* file.   

In [4]:
file = open('msc_links.txt','w')
for link in links:
	file.write(link[0]+'\n')

file.close()

Now, if we look at the file it's written in the correct way (:

## 1.2 Crawl master's degree pages 

In [62]:
def download_html(url, dest_folder):
    #path is the local path we're working in 
    if not os.path.exists(dest_folder):  # create folder if it does not exist
        os.makedirs(dest_folder) 
    
    filename = url.replace('https://www.findamasters.com//masters-degrees/course/', '').replace('/', '_')
    file_path = os.path.join(dest_folder, filename)
    url_req = requests.get(url)

    if url_req.ok:
        soup = BeautifulSoup(url_req.text, 'html.parser')
        with open(file_path, 'w') as file: 
            file.write(soup.prettify())
        
        file.close()
        

In [68]:
### SAVING HTML & ORGANIZING THEM IN FOLDERS ###
file = open('msc_links.txt', 'r')
lines_list = file.readlines()
file.close()

In [73]:
for i in range(1, 401):
    folder_i = 'page_' + str(i)
        
    #selecting the lines corresponding to the i-th page
    start = (i-1)*15
    stop =  i*15
    needed_lines = lines_list[start : stop]

    #saving the corresponding html in "page_i" folder
    for line in needed_lines:
        url  = 'https://www.findamasters.com/' + str(line)[:-1]
        download_html(str(url), folder_i)
        time.sleep(2)
    

## 1.3 Parse downloaded pages

Creating a DataSet starting from the scrapped *.tsv* files. 

In [2]:
msc_degrees = pd.DataFrame(columns=['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate', 'fees', 'modality', 'duration', 'city', 'country', 'administration', 'url'])
for i in range(1, 6001): 
    course_file = csv.reader(open(os.path.join('courses_tsv', 'course_'+str(i))+'.tsv', 'r'), delimiter='\t')
    line = pd.DataFrame(course_file, columns=['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate', 'fees', 'modality', 'duration', 'city', 'country', 'administration', 'url'], index = [i])
    msc_degrees = pd.concat([msc_degrees, line])


To see if we created the DataFrame in the correct way 

In [3]:
msc_degrees

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
1,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,School of Health and Life Sciences,Full time,Progress your career as a physiotherapist wit...,"January, September",Please see the university website for further...,MSc,1 Year Full Time / 2-3 Years Part Time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
2,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Economics and Management,Full time,Programme overview The Advanced Master in I...,September,18.000 €,MA MSc Other Pre-Masters Masters Module,10 months,Brussels,Belgium,On Campus,https://www.findamasters.com/masters-degrees/...
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wi...,September,Please see the university website for further...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
4,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stu...,See Course,Please see the university website for further...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound fina...,September,"UK: £18,000 (Total) International: £34,750 (...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time,Master of Science in Business (MSB) Our Mas...,See Course,Please see the university website for further...,MSc,12 months,Corvallis,USA,Online,https://www.findamasters.com/masters-degrees/...
5997,Material Culture & Artefact Studies - MSc/PgDip,University of Glasgow,College of Arts & Humanities,Full time,Material culture and artefact studies combine...,September,Please see the university website for further...,MSc PGDip,"9-12 months full-time, 18-24 months part-time",Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5998,Master's of Computer Science,Harbour.Space University,Masters Programmes,Full time,Harbour.Space’s Master's of Computer Science ...,"September, January","€29,900/year",MSc,1 year or 2 years,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...
5999,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Full time,Harbour.Space's FinTech Master programme ...,"September, January","€29,900/year",MBA MSc,1 Year,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...


# 2. Search Engine

## 2.0 Preprocessing

### 2.0.0  Preprocessing the text

We are required to preprocess all the informations retrieved for the courses, but since loosing punctuation in the url will lead us to loose its functionality, we won't preprocess that column (also, the info contained in the url aren't textual info). Moreover, we won't preprocess the fees column since there's a particoular preprocessing required for that column.

- 'description': removing punctuation and stopwords, stemming
- 'startDate': eliminating everithing that is not a month     (oss: calendar.month_name contains an empty name as first element)
- 'courseName': eliminating 'Msc' and similar: lo faccio dopo 


In [ ]:
# set to True to download 
if False:
    nltk.download('punkt') 
    nltk.download('stopwords')

In [4]:
#'descritpion' column 
stopw = stopwords.words('english')
msc_degrees['description_clean'] = msc_degrees.description.apply(lambda row: nltk.word_tokenize(row)).apply(lambda row: ([nltk.PorterStemmer().stem(word) for word in row if (word.isalnum() and (not word in stopw))])) 

#'startDate' column
msc_degrees['startDate_clean']= msc_degrees.startDate.apply(lambda row: nltk.word_tokenize(row)).apply(lambda row: ' '.join([month for month in row if month in calendar.month_name[1:]]))

#'courseName' column: TBD


In [12]:
msc_degrees

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,description_clean,startDate_clean
1,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,School of Health and Life Sciences,Full time,Progress your career as a physiotherapist wit...,"January, September",Please see the university website for further...,MSc,1 Year Full Time / 2-3 Years Part Time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[progress, career, physiotherapist, within, nh...",January September
2,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Economics and Management,Full time,Programme overview The Advanced Master in I...,September,18.000 €,MA MSc Other Pre-Masters Masters Module,10 months,Brussels,Belgium,On Campus,https://www.findamasters.com/masters-degrees/...,"[programm, overview, the, advanc, master, inno...",September
3,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wi...,September,Please see the university website for further...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[embark, profession, account, career, academ, ...",September
4,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stu...,See Course,Please see the university website for further...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[the, analyt, toxicolog, msc, uniqu, studi, co...",
5,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound fina...,September,"UK: £18,000 (Total) International: £34,750 (...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[busi, govern, reli, sound, financi, knowledg,...",September
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time,Master of Science in Business (MSB) Our Mas...,See Course,Please see the university website for further...,MSc,12 months,Corvallis,USA,Online,https://www.findamasters.com/masters-degrees/...,"[master, scienc, busi, msb, our, master, scien...",
5997,Material Culture & Artefact Studies - MSc/PgDip,University of Glasgow,College of Arts & Humanities,Full time,Material culture and artefact studies combine...,September,Please see the university website for further...,MSc PGDip,"9-12 months full-time, 18-24 months part-time",Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[materi, cultur, artefact, studi, combin, arch...",September
5998,Master's of Computer Science,Harbour.Space University,Masters Programmes,Full time,Harbour.Space’s Master's of Computer Science ...,"September, January","€29,900/year",MSc,1 year or 2 years,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...,"[master, comput, scienc, ii, root, mathemat, e...",September January
5999,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Full time,Harbour.Space's FinTech Master programme ...,"September, January","€29,900/year",MBA MSc,1 Year,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...,"[fintech, master, programm, design, prepar, gr...",September January


### 2.0.1 - Preprocessing the fees columns

First of all, we notice that there are a lot of courses for which we are addressed to the University's site to see fees' informations, and for these courses, th 'fees' columns says: *"Please see the university website for further information on fees for this course."*. For this reason, we replace such stirng with an empty one under the *'fees_clean'* column in order to do superflous operations.

In [41]:
msc_degrees['fees_clean'] = msc_degrees.fees.apply(lambda row: row.strip().replace('Please see the university website for further information on fees for this course.', ''))

In [121]:
len(msc_degrees[msc_degrees.fees_clean == ''])

4009

We notice that we now have 4009 empty lines for the fees values, which makes our operation actually useful.  
Now let's get started eith the rest of the pre-proessing of the fees column.

In [43]:
import re

In [185]:
currency_pattern = re.compile(u'([$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6|'r'USD|EUR|GBP|JPY|INR|AUD|CAD|HK][0-9]+[.,]{0,1}[0-9]+)|'r'([0-9]+[.,]{0,1}[0-9]+[USD|EUR|GBP|JPY|INR|AUD|CAD|HK|'u'[$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6])')
#msc_degrees['fees_clean'] = msc_degrees.fees_clean.apply(lambda row: nltk.word_tokenize(row))
msc_degrees['fees_clean'] = msc_degrees.fees_clean.apply(lambda row: re.findall(currency_pattern, row))
#msc_degrees['fees_clean'] = msc_degrees.fees_clean.apply(lambda row: re.split(currency_pattern, row))


In [186]:
aaaa = pd.DataFrame(msc_degrees.fees_clean)

In [153]:
len( msc_degrees[(msc_degrees.fees_clean.apply(len) > 0)])

1029

In [47]:
aiuto = pd.DataFrame(msc_degrees.fees_clean[msc_degrees.fees_clean != ''])
pattern2 = re.compile(u'([$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6]'r'|USD|EUR|GBP|JPY|INR|AUD|CAD|HK)')
#aiuto['fees_clean'] = aiuto.fees_clean.apply(lambda row: nltk.word_tokenize(row))
aiuto['fees_clean'] = aiuto.fees_clean.apply(lambda row: list(zip(re.findall(pattern2, row), re.findall(r'[0-9]+[.,]{0,1}[0-9]+', row)) ))

In [48]:
aiuto

2                    [(€, 18.000)]
5       [(£, 18,000), (£, 34,750)]
6                     [(€, 15000)]
9       [(£, 12,500), (£, 28,750)]
10                    [(€, 15000)]
                   ...            
5989        [(€, 7000), (€, 5750)]
5990                 [(€, 29,900)]
5995                 [(€, 29,900)]
5998                 [(€, 29,900)]
5999                 [(€, 29,900)]
Name: fees_clean, Length: 1991, dtype: object

In [49]:
len(aiuto[ (aiuto.fees_clean.apply(len) > 0) ])

1208

In [146]:
len(msc_degrees[ (msc_degrees['fees_clean'].apply(len) ==  0) ])

4972

In [80]:
len(msc_degrees[ (msc_degrees.fees_clean.apply(len)) > 0 ])


4580

In [81]:
msc_degrees['fees_clean'] 

1                       []
2              [18.000, €]
3                       []
4                       []
5       [£18,000, £34,750]
               ...        
5996                    []
5997                    []
5998        [€29,900/year]
5999        [€29,900/year]
6000                    []
Name: fees_clean, Length: 6000, dtype: object

## 2.1 Conjunctive query 

### 2.1.1 Create your index!

Creating *vocabulary* file.     
To do so, we first create a list, *vocabulary_list*, containing all the words in the '*description_clean*' column of the dataset, then we set each word's *term_id* as its implicit index in this list. To do so, zip the previous list with another explicitly containg the indeces we choose.
''', converted in string type (this way we don't have any problem when writing the *vocabulary.tsv* file).'''

In [5]:
# crating vocabulary lists as a tool to write the vocabulary file 
vocabulary_list = sorted(list(Counter(reduce(lambda x, y: x+y, msc_degrees.description_clean.values)).keys()))
vocabulary_idx = dict(zip(vocabulary_list, list(range( len(vocabulary_list)))))

In [6]:
# save dictionary to vocabulary.pkl file
with open('vocabulary.pkl', 'wb') as v_file:
    pickle.dump(vocabulary_idx, v_file)

v_file.close()

#### Inverted index creation

#va fatto con i file del vocabolario

In [7]:
# Read dictionary pkl file
with open('vocabulary.pkl', 'rb') as v_file:
    vocabulary_dict = pickle.load(v_file)

In [8]:
#function to get the documents in which term appears
get_documents_list = lambda word:  list(msc_degrees.loc[msc_degrees.description_clean.apply(lambda row: word in row )].index)

#initializing inverted_index_dict
inverted_index = dict()

#updating the inverted_index dictionary
for key in vocabulary_dict:
    #saving needed values in order to call the funcion only once per term 
    doc_list =  get_documents_list(key) 

    #updating dict
    inverted_index.update({vocabulary_dict[key] : doc_list})


In [9]:
inverted_index

{0: [107,
  117,
  150,
  193,
  334,
  471,
  516,
  525,
  534,
  613,
  655,
  683,
  686,
  716,
  756,
  854,
  863,
  867,
  891,
  903,
  975,
  1132,
  1141,
  1215,
  1346,
  1385,
  1441,
  1484,
  1717,
  1721,
  1724,
  1820,
  1834,
  1838,
  2082,
  2114,
  2265,
  2275,
  2294,
  2491,
  2589,
  2936,
  2959,
  3059,
  3116,
  3151,
  3257,
  3284,
  3530,
  3540,
  3572,
  3601,
  3661,
  3673,
  3722,
  3803,
  3877,
  3880,
  3882,
  3897,
  3972,
  3995,
  4197,
  4248,
  4249,
  4253,
  4258,
  4584,
  4860,
  4941,
  5006,
  5038,
  5124,
  5198,
  5252,
  5280,
  5467,
  5472,
  5502,
  5519,
  5542,
  5615,
  5683,
  5757,
  5784,
  5876,
  5923],
 1: [193,
  284,
  570,
  1132,
  1441,
  1484,
  1584,
  1695,
  1820,
  1834,
  1853,
  1854,
  2205,
  2265,
  2416,
  2491,
  2874,
  3191,
  3284,
  3572,
  3897,
  3995,
  4083,
  4084,
  4090,
  4108,
  4261,
  4584,
  4640,
  4995,
  5683,
  5793,
  5794,
  5816,
  5916,
  5964],
 2: [27,
  32,
  61,
  117,
  24

Storing the inverted index in a pkl file


In [11]:
# save dictionary to inverted_index.pkl file
with open('inverted_index.pkl', 'wb') as invidx_file:
    pickle.dump(inverted_index, invidx_file)

invidx_file.close()

### 2.1.2 Executing the query

First of all, we need to load the files for the query execution

In [10]:
# Read dictionary pkl file
with open('vocabulary.pkl', 'rb') as v_file:
    vocabulary_dict = pickle.load(v_file)

v_file.close()

In [12]:
#Read inverted_index pkl file
with open('inverted_index.pkl', 'rb') as invidx_file:
    inverted_index = pickle.load(invidx_file)

invidx_file.close()

- Now we have to preprocess the query in the same way we preprocessed the 'description' column to get a match: We define a function that does that, then we apply it to the query
- then we get the index of the documents_ids in which the words appear 
- return list of dovìcuments_ids

In [13]:
#getting the query from input and stripping it
query = input('Type to search:').strip()

#preprocessing the query
query = functions.preprocess_query(query)

#searching documents
doc_sat_query = functions.get_documents_conjunctive_query(query, vocabulary_dict, inverted_index)

Now we want to visualize the results given from the query

In [14]:
if doc_sat_query == []:
    print("This query didn't produce any result")
else: 
    print('Results:')

results_columns = ['courseName', 'universityName', 'description', 'url']
msc_degrees.loc[doc_sat_query, results_columns]

Results:


,courseName,universityName,description,url
2048,Clinical Optometry - MSc,Cardiff University,Why study this course The aim of this prog...,https://www.findamasters.com/masters-degrees/...
4098,Global Meetings and Events Management MSc / PGDip,University College Birmingham,Become part of an events industry worth an es...,https://www.findamasters.com/masters-degrees/...
4,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stu...,https://www.findamasters.com/masters-degrees/...
5,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound fina...,https://www.findamasters.com/masters-degrees/...
8,Addictions MSc,King’s College London,Join us for an online session for prospective...,https://www.findamasters.com/masters-degrees/...
...,...,...,...,...
1991,Clinical Embryology - MSc/PGDip,University of Leeds,The Leeds MSc in Clinical Embryology is a par...,https://www.findamasters.com/masters-degrees/...
1999,Clinical Geriatrics - MSc,Cardiff University,Why study this course The MSc Clinical Ger...,https://www.findamasters.com/masters-degrees/...
2021,Clinical Neuropsychology - MSc,University of Bristol,Professional programmes in Clinical Neuropsyc...,https://www.findamasters.com/masters-degrees/...
2029,Clinical Ophthalmic Practice MSc,University College London,Register your interest in graduate study at U...,https://www.findamasters.com/masters-degrees/...


In [15]:
res = msc_degrees.loc[doc_sat_query, results_columns]
names = ['Analytical Toxicology MSc', 'Addictions MSc', 'Accounting and Finance - MSc', 'Allergy - MSc/PGDip/PGCert']
res[res['courseName'].isin(names)]

,courseName,universityName,description,url
4,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stu...,https://www.findamasters.com/masters-degrees/...
5,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound fina...,https://www.findamasters.com/masters-degrees/...
8,Addictions MSc,King’s College London,Join us for an online session for prospective...,https://www.findamasters.com/masters-degrees/...
528,Accounting and Finance - MSc,University of Sussex,On this MSc you’ll advance your accounting an...,https://www.findamasters.com/masters-degrees/...
1012,Allergy - MSc/PGDip/PGCert,Imperial College London,Allergy is an increasing global health proble...,https://www.findamasters.com/masters-degrees/...


Sembra tutto giusto, domnani sistemo e faccio quello con lo score eheh